In [10]:
import hashlib
import binascii
import os

### Helper functions

In [111]:
def Integer_to_bytes(s, nbytes):
    return int.to_bytes(int(s), nbytes, "little")

def Integer_from_bytes(s):
    return Integer(int.from_bytes(s, "little"))
# Adds the two points P and Q and gives the result
# in the coordinates
def point_add(P, Q):
    A = (P[1] - P[0]) * (Q[1] - Q[0]) % p
    B = (P[1] + P[0]) * (Q[1] + Q[0]) % p
    C = 2 * P[3] * Q[3] * d % p
    D = 2 * P[2] * Q[2] % p
    E, F, G, H = B - A, D - C, D + C, B + A
    return (E * F, G * H, F * G, E * H)
# Multiplies s with point P and gives the result
# in the coordinates
def point_mul(s, P):
    Q = (0, 1, 1, 0)
    for bit in Integer(s).bits():
        if bit:
            Q = point_add(Q, P)
        P = point_add(P, P)
    return Q
# recovers x-coordinates of a point on the curve
# from the y-coordinates.
def recover_x(y, sign):
    x2 = (y^2-1) * (d*y^2+1)^-1
    x = sqrt(x2)
    if Integer(x).bits()[0] != sign:
        return -x
    else:
        return x
# returns hashed secret key and the upper 32
# bytes of the sha512 hash
def secret_expand(S):
    h = hashlib.sha512(S).digest()
    a = Integer_from_bytes(h[:32])
    a &= 2^254 - 8
    a |= 2^254
    return (a, h[32:])

def point_equal(P, Q):
    if (P[0] * Q[2] - Q[0] * P[2]) % p != 0:
        return False
    if (P[1] * Q[2] - Q[1] * P[2]) % p != 0:
        return False
    return True

## Point addition

In [15]:
p = 2**255 - 19
d = -121665 * pow(121666, p - 2, p) % p
P = (9, 5, 1, 1)
Q = (4, 8, 1, 1)
result = point_add(P, Q, d, p)
print(f"Point addition {result}")

Point addition (12246746044957381706561499803493136934360282763281958624921836758123273160092, 45261948119838604227591840818660755853614106905203869154805153290552271130918, 4639372086675119516775102087422346591749431038861670617759518883504447227352, 27968)


## Initialization

In [112]:
message = "Crypto"
m = message.encode()
p = 2**255 - 19
GFp = GF(p)
d = GFp(-121665 * 121666**-1)
q = 2**252 + 27742317777372353535851937790883648493

In [113]:
g_y = GFp(4 * 5**-1)
g_y = "0x" + str(g_y)
g_y

'0x46316835694926478169428394003475163141307993866256225615783033603165251855960'

In [114]:
g_y = GFp(4 * 5**-1)

In [115]:
g_x = recover_x(g_y, 0)
g_x = "0x" + str(g_x)
g_x

'0x15112221349535400772501151409588531511454012693041857206046113283949847762202'

In [116]:
g_x = recover_x(g_y, 0)

In [117]:
G = (g_x, g_y, 1, g_x * g_y % p)
G

(15112221349535400772501151409588531511454012693041857206046113283949847762202,
 46316835694926478169428394003475163141307993866256225615783033603165251855960,
 1,
 46827403850823179245072216630277197565144205554125654976674165829533817101731)

## Key Generation

In [1]:
!pip install cryptography

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 6.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [56]:
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import serialization
import hashlib
sk = x25519.X25519PrivateKey.generate()
print(sk)
pk = sk.public_key()
pk_bytes = pk.public_bytes(
    encoding=serialization.Encoding.Raw,
    format=serialization.PublicFormat.Raw
)
print("0x" + pk_bytes.hex())

0xd486c40246503e35f5066700d7845ab40e62f85196efcb4a130f00461212d560


In [54]:
def point_compress(pk):
    return pk.public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw,
    )

## Signature and Verification

In [55]:
def signature(message):
    prefix = b'Crypto'
    h1 = hashlib.sha512(prefix + message).digest()
    # sha512 hash was successful
    # Compute the integer from the hash
    R = int.from_bytes(h1, byteorder="little") % (2**255 - 19)
    nonce = os.urandom(32)
    r = hashlib.sha512(nonce + message).digest()
    # Compress the r point
    r_bytes = point_compress(pk)
    # Hash with sha512 again, with compressed point and message
    h2 = hashlib.sha512(r_bytes + r + message).digest()
    h_int = int.from_bytes(h2, byteorder="little") % (2**255 - 19)
    # Compute the signature
    s = (R + h_int * int.from_bytes(r, "little")) % (2**255 - 19)
    # verification
    signature = (r_bytes, s.to_bytes(32, "little"))
    # Print signature
    return r_bytes, s.to_bytes(32, "little")

m = b"Crypto"
r, s, = signature(m)
print(f"r = 0x{r.hex()}")
print(f"s = 0x{s.hex()}")
print(len(r))
print(len(s))

r = 0xc3f5439ac9b37a2310187e702d22ee7803d26a7644ea5c7a9d65bdfb0bb1e55c
s = 0x220a280488c53860f7c144c00ff55a41e8121221b2370e1962f100da6fd00273
32
32


## Points and Signature values

**Base Point (G)**:

- G_x = 0x15112221349535400772501151409588531511454012693041857206046113283949847762202 
- G_y = 0x46316835694926478169428394003475163141307993866256225615783033603165251855960
  
**Private Key (sk)**: 
    - \( sk = cryptography.hazmat.bindings._rust.openssl.x25519.X25519PrivateKey object at 0x7f20780c2eb0 \)
  
**Public Key (pk)**: 
    - \( pk = 0xd486c40246503e35f5066700d7845ab40e62f85196efcb4a130f00461212d560 \)

### Signature:
- \( r = 0xc3f5439ac9b37a2310187e702d22ee7803d26a7644ea5c7a9d65bdfb0bb1e55c \)
- \( s = 0x220a280488c53860f7c144c00ff55a41e8121221b2370e1962f100da6fd00273 \)